Reliability Models for Facility Location:The Expected Failure Cost Case
---

Base on the following article:

*Snyder, L. V., & Daskin, M. S. (2005). Reliability models for facility location: the expected failure cost case. Transportation Science, 39(3), 400-416.*

In [6]:
from docplex.mp.model import Model

In [7]:
I = range(10)
J = range(10)
c = [[1 for i in I] for j in J]
f = [1 for j in J]

In [13]:
###################
# create one model instance
m = Model(name='UFLP')

###################
# Define variables

# x(i,j) equals 1 if arc ij is in the solution
X = {
    (i,j): m.binary_var(name='X_%d_%d' % (i,j))
     for i in I for j in J
} 
# y(j) equales 1 if node j is in the solution
Y = {
    (j): m.binary_var(name='Y_%d}' % (j))
    for j in J
}

###################
# Define Objective
m.minimize(m.sum(X[i,j] * c[i][j] for i in I for j in J) \
          + m.sum(Y[j] * f[j] for j in J))

###################
# Define constraints

# constraint #1: each customer is affected to a facility
for j in J:
    m.add_constraint(m.sum(X[i,j] for i in I) == 1, ctname='demande_%s' % j)

# constraint #2: customer can only be associated to open facilities
for i in I:
    for j in J:
        m.add_constraint(X[i,j] <= Y[j], ctname='flow_%s_%s' % (i,j))



In [14]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python is present, version is 3.6.5
* docplex is present, version is (2, 4, 61)
* CPLEX wrapper is present, version is 12.8.0.0, located at: /opt/conda/lib/python3.6/site-packages


In [15]:
m.print_information()

Model: UFLP
 - number of variables: 110
   - binary=110, integer=0, continuous=0
 - number of constraints: 110
   - linear=110
 - parameters: defaults
